In [1]:
#!pip install pandas
import pandas as pd
import numpy as np
import random

In [2]:
filename = "mnist_test.csv"
df = pd.read_csv(filename, header=None)
if filename == "mnist_test.csv":
    df_column_y = 0
    df = df.iloc[:200]
    
elif filename == "iris.csv":
    df_column_y = 4
df.head()


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Get 20% of dataframe as test

In [3]:
X_test_size = int(df.shape[0] * 0.2)
X_i = set()
while len(X_i) != X_test_size:
    X_i.add(random.randint(0, df.shape[0]-1))
X_i = list(X_i)

In [4]:
df_test = df.iloc[X_i]
X_test = np.array(df_test.drop(df_column_y, axis=1))
y_test = np.array(df_test[df_column_y])

## Get the remaining train dataframe

In [5]:
df_train = df.drop(X_i)
X_train = np.array(df_train.drop(df_column_y, axis=1))
y_train = np.array(df_train[df_column_y])
X_train.shape

(160, 784)

## Fit and predict using custom model

In [6]:
from KNN import KNearestNeighbors

my_knn_model = KNearestNeighbors(1, distance_metric="euclidean")
my_knn_model.fit(X=X_train, y=y_train)
y_pred = my_knn_model.predict(X_test)

## Analyze results

In [7]:
def getAcc(y_test, y_pred):
    df_results = pd.DataFrame(columns=["Real","Predict"])
    df_results.Real = y_test
    df_results.Predict = y_pred
    
    count = 0
    for i in range(df_results.shape[0]):
        experiment = df_results.iloc[i]
        if experiment.Real == experiment.Predict:
            count += 1

    accuracy = (count/df_results.shape[0]) * 100
    return accuracy

In [8]:
accuracy = getAcc(y_test, y_pred)

print(f"Accuracy at {accuracy}%")

Accuracy at 77.5%


In [9]:
#df_results[df_results.Real != df_results.Predict]

## Parameter configuring

In [11]:
k_list = list(range(1,5))
distances = ["euclidean", "manhattan", "cosine"]
weights = ["uniform","distance"]
results = []

for ks in k_list:
    for d in distances:
        for w in weights:
            my_knn_model = KNearestNeighbors(ks, distance_metric=d, weight=w)
            my_knn_model.fit(X=X_train, y=y_train)
            y_pred = my_knn_model.predict(X_test)
            acc  = getAcc(y_test, y_pred)
            results.append([ks,d,w,acc])
            
results = sorted(results, reverse=True, key=lambda x: x[3])
for x in results[:5]:
    #print(x)    
    print(f"Neighboor count: {x[0]:<5} Distance metric:{x[1]:15} Weight method:{x[2]:15} = Accuracy:{x[3]:}%")

Neighboor count: 3     Distance metric:cosine          Weight method:uniform         = Accuracy:90.0%
Neighboor count: 3     Distance metric:cosine          Weight method:distance        = Accuracy:87.5%
Neighboor count: 4     Distance metric:cosine          Weight method:uniform         = Accuracy:87.5%
Neighboor count: 4     Distance metric:cosine          Weight method:distance        = Accuracy:87.5%
Neighboor count: 1     Distance metric:cosine          Weight method:uniform         = Accuracy:85.0%
